In [2]:
import pandas as pd
import numpy as np
from numba import njit
import os
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
sample_arr = pd.read_csv("annovar-full/chr1_22251.txt", usecols=[1], dtype={"Start":"int"}, sep='\t').to_numpy().ravel()
print(len(sample_arr), sample_arr)

81996 [    65505     65521     68992 ... 248917046 248917114 248917151]


In [4]:
# searchsort but filters the indexes it couldnt find
@njit
def db_search(database, value):
    idx = np.searchsorted(database, value)
    if database[idx] == value:
        return idx
    else:
        return -1

In [28]:
chunksize = 10 ** 6
dtypes={
    "Start": "int",
    "End": "int",
    "Ref": "string",
    "Alt": "string",
    "Value": "string"
}
outputs = []
for chunk in pd.read_csv("databases/hg38_avsnp150.txt", names=["Start","End","Ref","Alt","Value"], dtype=dtypes, index_col=0, sep='\t', 
                         usecols=[1,2,3,4,5], na_filter=False, nrows=27_783_731, chunksize=chunksize):
    output_idxs = []
    db_arr = chunk.index.to_numpy()
    # where the chunks first and last values are in the sample array
    start = np.searchsorted(sample_arr, db_arr[0])
    end = np.searchsorted(sample_arr, db_arr[-1])
    # TODO: CHECK INDEXING OF CURR_ARR - END + 1?
    curr_arr = sample_arr[start:end]
    for x in curr_arr:
        output_idxs.append(db_search(db_arr, x))
    for x in output_idxs:
        # output idxs -1 == db_search return N/A
        if x >= 0:
            outputs.append(chunk["Value"].iloc[x])
        else:
            outputs.append(".")
    # SAVETXT WONT DO IT ITERATIVELY - NEED FUNCTION
    # np.savetxt("avsnp_22251.csv", outputs, delimiter="\t")
    # if last val of chunk is greater than input arrays last val, stop looping chunks unecessary
    if end == len(sample_arr):
        break

In [31]:
# n = nrows, s = skiprows, csize = chunksize, db name must be inside database folder
# assumes the fifth column will be the value wanted to retrieve
def annotate_database(db_name, n, s=0, csize = 10 ** 5, value_type="float"):
    # cancels the s + 1 in skiprows parameter
    if s > 0:
        s -= 1
    chunksize = csize
    dtypes={
        "Start": "int",
        "End": "int",
        "Ref": "string",
        "Alt": "string",
        "Value": value_type
    }
    outputs = []
    # s + 1 to avoid headers from first chromosome
    for chunk in pd.read_csv("databases/%s.txt" % db_name, names=["Start","End","Ref","Alt","Value"], dtype=dtypes, index_col=0, sep='\t', 
                             usecols=[1,2,3,4,5], na_filter=False, skiprows=s + 1, nrows=n, chunksize=chunksize):
        output_idxs = []
        db_arr = chunk.index.to_numpy()
        # where the chunks first and last values are in the sample array
        start = np.searchsorted(sample_arr, db_arr[0])
        end = np.searchsorted(sample_arr, db_arr[-1])
        # TODO: CHECK INDEXING OF CURR_ARR - END + 1?
        curr_arr = sample_arr[start:end]
        for x in curr_arr:
            output_idxs.append(db_search(db_arr, x))
        for x in output_idxs:
            # output idxs -1 == db_search return N/A
            if x >= 0:
                outputs.append(chunk["Value"].iloc[x])
            else:
                outputs.append(".")
        # SAVETXT WONT DO IT ITERATIVELY - NEED FUNCTION
        # np.savetxt("avsnp_22251.csv", outputs, delimiter="\t")
        # if last val of chunk is greater than input arrays last val, stop looping chunks unecessary
        if end == len(sample_arr):
            break
    return outputs

In [32]:
# annotate_database("hg38_exac03", 1_089_538)
func_output = annotate_database("hg38_avsnp150", 27_783_731, value_type="string")

In [40]:
print(func_output[10 ** 4 - 1] == outputs[10 ** 4])

True


In [ ]:
np.savetxt('chunkv2test_avsnp.csv', outputs, delimiter=',', fmt="%s")

In [138]:
main_df = pd.read_csv("alldb_mitch_chr1.csv", sep=',', usecols=[2], squeeze=True)
test_inp = main_df.to_numpy()
print(len(test_inp))
# test_inp = [10031, 10037, 10043, 10055, 10057, 10061, 10061, 10061, 10064, 10064, 10067, 10067, 10108, 10108, 10108, 10108, 10109, 10108, 10110, 10109, 10111, 10114, 10113, 10114, 10114, 10115, 10114, 10115, 10114, 10116, 10117, 10116, 10117, 10116, 10118, 10117, 10118, 10120, 10119, 10120, 10120, 10120, 10121, 10120, 10122, 10122, 10123, 10122, 10124, 10123, 10126, 10125, 10126, 10126]
# print(test_inp)

3186


In [77]:
chunksize = 10 ** 6
dtypes={
    "Start": "int",
    "End": "int",
    "Ref": "string",
    "Alt": "string",
    "Value": "string"
}
outputs_test = []
for chunk in pd.read_csv("databases/gnomad/gnomad30.txt", names=["Start","End","Ref","Alt","Value"], dtype=dtypes, index_col=0, sep='\t', 
                         usecols=[1,2,3,4,5], na_filter=False, skiprows=1, nrows=63_500_523, chunksize=chunksize):
    output_idxs = []
    db_arr = chunk.index.to_numpy()
    # where the chunks first and last values are in the sample array
    start = np.searchsorted(test_inp, db_arr[0])
    end = np.searchsorted(test_inp, db_arr[-1])
    # TODO: CHECK INDEXING OF CURR_ARR - END + 1?
    curr_arr = test_inp[start:end]
    for x in curr_arr:
        output_idxs.append(db_search(db_arr, x))
    for x in output_idxs:
        # output idxs -1 == db_search return N/A
        if x >= 0:
            outputs_test.append(chunk["Value"].iloc[x])
        else:
            outputs_test.append(".")
    # SAVETXT WONT DO IT ITERATIVELY - NEED FUNCTION
    # np.savetxt("avsnp_22251.csv", outputs_test, delimiter="\t")
    # if last val of chunk is greater than input arrays last val, stop looping chunks unecessary
    if end == len(test_inp):
        break

In [ ]:
np.savetxt('gnomadgen_output3.csv', outputs_test, delimiter=',', fmt="%s")

In [169]:
test_out = [-1] * len(test_inp)

chunksize = 10 ** 6
dtypes={
    "Start": "int",
    "End": "int",
    "Ref": "string",
    "Alt": "string",
    "AF": "float"
}
# 63_500_523
curr_inp_idx = 0
for chunk in pd.read_csv("databases/gnomad/gnomad30.txt", 
                         dtype=dtypes, index_col=0, sep='\t', 
                         usecols=[1,2,3,4,5], na_filter=True, na_values=".",
                         nrows=63_500_523, chunksize=chunksize):
    # test_inp end value not start value ideally
    for idx, val in enumerate(test_inp):
        try:
            value = chunk.loc[val]
        except (KeyError) as error:
            continue
        else:
            if type(value) == pd.core.frame.DataFrame:
                test_out[idx] = value.iloc[0][-1]
            else:
                test_out[idx] = value[-1]

In [172]:
print(len(test_out))
print(test_out.index(.0000279))
print(test_out)

3186
788
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0.0004, -1, -1, -1, -1, -1, 0.0003, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0.0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0.0001, -1, 6.993e-06, 0.0001, 0.0001, -1, -1, -1, 6.985e-06, -1, -1, -1, -1, -1, -1, -1, -1, 0.0009, -1, -1, -1, -1, -1, -1, -1, 0.002, 0.002, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -

In [173]:
np.savetxt('inp_validation.csv', test_inp, delimiter=',', fmt="%s")